# Assimilating sparse data

In this demo, we'll look once again at estimating the fluidity coefficient $A$ in Glen's flow law

$$\dot\varepsilon = A\tau^3$$

from observational data for the Larsen Ice Shelf.
The [previous tutorial](https://icepack.github.io/notebooks/tutorials/05-ice-shelf-inverse/) showed how to solve statistical estimation problems.
Here we'll explore how to regularize those estimation problems.
We'll again use the reparameterization trick of inferring the field $\theta$ in

$$A = A_0e^\theta$$

in order to guarantee that the fluidity coefficient is positive.
Before, we used the regularization functional

$$R(\theta) = \frac{L^2}{2\Theta^2}\int_\Omega|\nabla \theta|^2\mathrm dx.$$

This functional penalizes sharp gradients in the inferred field.
For some problems, however, we might want to instead penalize the *curvature* or the second derivative of the inferred field instead:

$$R(\theta) = \frac{L^4}{2\Theta^2}\int_\Omega|\nabla^2\theta|^2\mathrm dx.$$

Note how we have a factor of $L^4$ in order to get the units right now instead of $L^2$.
But there's a problem.
Say we were using a conventional piecewise linear finite element basis.
These functions are differentiable, but their derivatives are discontinuous across cell boundaries.
We can discretize problems involving first but not second or higher-order derivatives.
There are a few ways we can dig ourselves out.
Here we'll show one approach -- using an additional field to represent the curvature.

### Input data

The input data are the same as from the previous demo on inferring the fluidity of the Larsen Ice Shelf.

In [ ]:
import geojson
import firedrake
import icepack

outline_filename = icepack.datasets.fetch_outline("larsen-2015")
with open(outline_filename, "r") as outline_file:
    outline = geojson.load(outline_file)

icepack.meshing.collection_to_gmsh(outline).write("larsen.msh")
mesh = firedrake.Mesh("larsen.msh")

The code below is the same boilerplate as in the previous tutorial for plotting simulation results on top of the mosaic of Antarctica image.

In [ ]:
import numpy as np
import rasterio
import icepack.plot

coords = np.array(list(geojson.utils.coords(outline)))
δ = 5e3
xmin, xmax = coords[:, 0].min() - δ, coords[:, 0].max() + δ
ymin, ymax = coords[:, 1].min() - δ, coords[:, 1].max() + δ

image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    image_window = rasterio.windows.from_bounds(
        left=xmin,
        bottom=ymin,
        right=xmax,
        top=ymax,
        transform=image_file.transform,
    )
    image = image_file.read(indexes=1, window=image_window, masked=True)


def subplots(*args, **kwargs):
    fig, axes = icepack.plot.subplots(*args, **kwargs)
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(
        image_window, image_file.transform
    )
    kw = {
        "extent": (xmin, xmax, ymin, ymax),
        "cmap": "Greys_r",
        "vmin": 12e3,
        "vmax": 16.38e3,
    }
    try:
        axes.imshow(image, **kw)
    except AttributeError:
        for ax in axes:
            ax.imshow(image, **kw)

    return fig, axes

In [ ]:
fig, axes = subplots()
axes.set_xlabel("meters")
kwargs = {
    "interior_kw": {"linewidth": 0.25},
    "boundary_kw": {"linewidth": 2},
}
firedrake.triplot(mesh, axes=axes, **kwargs)
axes.legend(loc="upper right");

Just like in the previous demos, we'll apply a smoothing filter to the thickness, which is necessary to get a reasonable driving stress.

In [ ]:
import xarray
from firedrake import assemble, Constant, avg, jump, dot, inner, grad, dx, ds, dS

thickness_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness = xarray.open_dataset(thickness_filename)["thickness"]

Q = firedrake.FunctionSpace(mesh, family="CG", degree=2)
h0 = icepack.interpolate(thickness, Q)

h = h0.copy(deepcopy=True)
α = Constant(2e3)
J = 0.5 * ((h - h0) ** 2 + α ** 2 * inner(grad(h), grad(h))) * dx
F = firedrake.derivative(J, h)
firedrake.solve(F == 0, h)

Read the velocity data and uncertainties.

In [ ]:
velocity_filename = icepack.datasets.fetch_measures_antarctica()
velocity_dataset = xarray.open_dataset(velocity_filename)
vx = velocity_dataset["VX"]
vy = velocity_dataset["VY"]
errx = velocity_dataset["ERRX"]
erry = velocity_dataset["ERRY"]

V = firedrake.VectorFunctionSpace(mesh, family="CG", degree=2)
u_obs = icepack.interpolate((vx, vy), V)
σx = icepack.interpolate(errx, Q)
σy = icepack.interpolate(erry, Q)

Here's where things start to get different.
Before, we took the fluidity parameter $\theta$ to live in some scalar function space.
Here we'll instead work with *both* the fluidity and an additional variable $\kappa$ for curvature.
The constraint between the two fields is that
$$\kappa = \nabla^2\theta.$$
In order to simultaneously infer both fields, we need to make a function space that will store both.

We'll use the same value initial guess for $\theta$ as in the second demo -- a constant fluidity assuming a temperature of -13C.
This makes the initial curvature equal to zero.

In [ ]:
Σ = firedrake.FunctionSpace(mesh, "HHJ", degree=1)
Z = Q * Σ
z = firedrake.Function(Z)

There's going to be another difference in how we define the viscosity function.
Before, we took in the logarithm of the fluidity as a keyword argument.
Now the fluidity and its curvature come packaged together.
The method `.split` will pull them apart.

In [ ]:
T = Constant(260)
A0 = icepack.rate_factor(T)

def viscosity(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    z = kwargs["log_fluidity_curvature"]
    θ, κ = firedrake.split(z)

    A = A0 * firedrake.exp(θ)
    return icepack.models.viscosity.viscosity_depth_averaged(
        velocity=u, thickness=h, fluidity=A
    )

The setup of the model is the same as before.

In [ ]:
model = icepack.models.IceShelf(viscosity=viscosity)
opts = {
    "dirichlet_ids": [2, 4, 5, 6, 7, 8, 9],
}
solver = icepack.solvers.FlowSolver(model, **opts)

u = solver.diagnostic_solve(
    velocity=u_obs,
    thickness=h,
    log_fluidity_curvature=z,
)

### Inferring the fluidity

There are four parts that go into an inverse problem:

* a physics model
* an initial guess for the parameter and state
* an error metric
* a smoothness metric

We already have the physics model and some initial guesses.
The physics are wrapped up in the Python function `simulation` defined below; we'll pass this function when we create the inverse problem.

In [ ]:
def simulation(z):
    return solver.diagnostic_solve(
        velocity=u_obs, thickness=h, log_fluidity_curvature=z
    )

Since the ice shelf is so large, we're going to want to scale some of our output quantities of interest by the area of the shelf.
This will make everything into nice dimensionless numbers, rather than on the order of $10{}^{10}$.

In [ ]:
area = assemble(Constant(1.0) * dx(mesh))
Ω = Constant(area)

The next step is to write a pair of Python functions that will create the model-data misfit functional and the regularization functional.
The key difference here is in how we create the regularization functional $R$.
This will encode both the penalty on high curvature as well as the constraint that $\kappa$ really is the curvature of $\theta$:
$$R = \frac{L^4}{|\Omega|}\int_\Omega\left(\frac{1}{2}|\kappa^2| - \kappa :\nabla^2\theta\right)dx.$$
This expression would be enough if the finite element basis that we used for $\theta$ had enough derivatives, which it doesn't.
What makes HHJ elements for the curvature so convenient is that there are only a couple of extra terms to fix this problem.
These involve jumps in the normal derivative of $\theta$.
What they don't involve is funny mesh-dependent constants that you have to tweak.
The correction terms are:
$$\ldots + \sum_\gamma\int_\gamma \text{avg}(\nu\cdot\kappa\nu)\cdot \text{jump}(\nabla\theta\cdot\nu)\,d\gamma$$
where the sum is over all edges of the mesh.
You can read more about HHJ elements and how to derive these correction terms [here](https://shapero.xyz/posts/plate-theory/).

In [ ]:
def loss_functional(u):
    δu = u - u_obs
    return 0.5 / Ω * ((δu[0] / σx)**2 + (δu[1] / σy)**2) * dx


def regularization(z):
    Θ = Constant(1.)
    L = Constant(125.0)
    Λ = L**4 / Θ**2 / Ω

    θ, κ = firedrake.split(z)
    ν = firedrake.FacetNormal(mesh)
    R_1 = (0.5 * inner(κ, κ) - inner(κ, grad(grad(θ)))) * dx
    κ_νν = inner(ν, dot(κ, ν))
    R_2 = avg(κ_νν) * jump(grad(θ), ν) * dS
    R_3 = κ_νν * inner(grad(θ), ν) * ds

    return L**4 / (Θ**2 * Ω) * (R_1 + R_2 + R_3)

Now we'll create create a `StatisticsProblem` object.
To specify the problem, we need to give it a procedure for running the simulation, another procedure for evaluating how good our guess is, and an initial guess for the unknown parameters.

In [ ]:
from icepack.statistics import StatisticsProblem, MaximumProbabilityEstimator

problem = StatisticsProblem(
    simulation=simulation,
    loss_functional=loss_functional,
    regularization=regularization,
    controls=z,
)

Now that we've specified the problem, we'll create a `MaximumProbabilityEstimator` to look for a solution.
The runtime is about the same as in the previous demo, so feel free to put on a fresh pot of coffee.

In [ ]:
estimator = MaximumProbabilityEstimator(
    problem,
    gradient_tolerance=1e-4,
    step_tolerance=1e-1,
    max_iterations=50,
)
z = estimator.solve()

As before, the algorithm reduces the objective by two orders of magnitude by the time it's converged.
The computed log-fluidity field looks very similar to the one obtained when we matched the computed velocity to the field obtained by interpolating the observations to the mesh.

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(z.subfunctions[0], vmin=-5, vmax=+5, axes=axes)
fig.colorbar(colors);